In [2]:
import pandas as pd
import numpy as np
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
from tpot import TPOTClassifier
import joblib
from eli5.sklearn import PermutationImportance

In [16]:
tpot_data = pd.read_csv('./game_predict_10_player_from0605.csv')
tpot_data = tpot_data.drop(columns = ['season_id', 'game_id', 'wl_away',
                                      'team_name_home', 'team_name_away', 
                                      'order_rate_home', 'order_rate_away',
                                      'video_available_rate_home','video_available_rate_away'])

tpot_data['All_nba_3rd_team_players_1yr_away'] = tpot_data['All_nba_3rd_team_players_1yr_away'].apply(lambda x: 0 if x < 0 else x)
X = tpot_data.drop(columns=['wl_home'])
y = tpot_data['wl_home']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=71)
# train_prop = 0.8
# num = round(len(X)*train_prop)
# X_train = X[:num]
# X_test = X[num:]
# y_train = y[:num]
# y_test = y[num:]

In [19]:
# train = tpot_data[:num]
# test = tpot_data[num:]

In [24]:
pipeline_optimizer = TPOTClassifier(generations=300, population_size=100, warm_start=True, 
                                    cv=10, max_time_mins=300, n_jobs = 4, verbosity=2)

In [25]:
pipeline_optimizer.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.6769177497629105

Generation 2 - Current best internal CV score: 0.6782394666711798

Generation 3 - Current best internal CV score: 0.6782394666711798

Generation 4 - Current best internal CV score: 0.6783561074176807


TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: LinearSVC(RobustScaler(Normalizer(Normalizer(input_matrix, norm=l2), norm=l1)), C=20.0, dual=False, loss=squared_hinge, penalty=l1, tol=1e-05)


TPOTClassifier(cv=10, generations=300, max_time_mins=300, n_jobs=4, verbosity=2,
               warm_start=True)

In [26]:
print(pipeline_optimizer.score(X_test, y_test))

0.6630384077126419


In [27]:
pipeline_optimizer.fitted_pipeline_

Pipeline(steps=[('normalizer-1', Normalizer()),
                ('normalizer-2', Normalizer(norm='l1')),
                ('robustscaler', RobustScaler()),
                ('linearsvc',
                 LinearSVC(C=20.0, dual=False, penalty='l1', tol=1e-05))])

In [28]:
pipeline_optimizer.export('tpot_exported_pipeline_new0606.py')

In [29]:
!cat tpot_exported_pipeline_new0606.py

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer, RobustScaler
from sklearn.svm import LinearSVC

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

# Average CV score on the training set was: 0.6783561074176807
exported_pipeline = make_pipeline(
    Normalizer(norm="l2"),
    Normalizer(norm="l1"),
    RobustScaler(),
    LinearSVC(C=20.0, dual=False, loss="squared_hinge", penalty="l1", tol=1e-05)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [32]:
# NOTE: Make sure that the outcome column is labeled 'target' in the data file
from sklearn.svm import LinearSVC
from sklearn.preprocessing import Normalizer, RobustScaler
tpot_data = pd.read_csv('./game_predict_10_player_from0605.csv')
tpot_data = tpot_data.drop(columns = ['season_id', 'game_id', 'wl_away',
                                      'team_name_home', 'team_name_away', 
                                      'order_rate_home', 'order_rate_away',
                                      'video_available_rate_home','video_available_rate_away'])
tpot_data['All_nba_3rd_team_players_1yr_away'] = tpot_data['All_nba_3rd_team_players_1yr_away'].apply(lambda x: 0 if x < 0 else x)
X = tpot_data.drop(columns=['wl_home'])
y = tpot_data['wl_home']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Average CV score on the training set was: 0.6783562736371663
exported_pipeline = make_pipeline(
    Normalizer(norm="l2"),
    Normalizer(norm="l1"),
    RobustScaler(),
    LinearSVC(C=20.0, dual=False, loss="squared_hinge", penalty="l1", tol=1e-05)
)


exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)

0.6780840558298666


/home/lytt/miniconda3/envs/pyflask/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [22]:
accuracy_score(y_test, results)

0.6753226558855544

In [26]:
filename = "tpot0606.joblib"

# save model
joblib.dump(exported_pipeline, filename)

# load model
loaded_model = joblib.load(filename)